In [1]:
import pandas as pd
import sqlite3
import warnings
import re
from datetime import datetime
warnings.filterwarnings(action='ignore')

In [ ]:
# import os
# os.environ["RUST_LOG"]="connectorx=debug,connectorx_python=debug"

In [ ]:
# import connectorx as cx
# import urllib

In [ ]:
# conn = cx.connect("../db/hr.db")

In [ ]:
# conn

In [2]:
query = "SELECT 자료생성년월, 사업장명, 사업자등록번호, 사업장업종코드명, 가입자수, 신규취득자수, 상실가입자수, 당월고지금액 FROM pension"

In [4]:
# res = cx.read_sql(conn, query, return_type="arrow")
# df = res.to_pandas(split_blocks=False, date_as_object=False)
# df.head()

In [ ]:
# %%timeit
# df = cx.read_sql(conn, query, partition_on="자료생성년월", partition_num=4)
# df.head()

In [5]:
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_columns', 1000)

In [6]:
con = sqlite3.connect("../db/hr.db")
df = pd.read_sql_query("SELECT 자료생성년월, 사업장명, 사업자등록번호, 사업장업종코드명, 가입자수, 신규취득자수, 상실가입자수, 당월고지금액  from pension", con)

In [7]:
df['자료생성년월'].unique()

array(['2023-01', '2022-12', '2022-11', '2022-10', '2022-09', '2022-08',
       'Jul-22', 'Jun-22', 'May-22', 'Apr-22', 'Mar-22', 'Feb-22',
       'Jan-22', '2023-02', '2023-03', 'Jan-21', 'Feb-21', 'Mar-21',
       'Apr-21', 'May-21', 'Jun-21', 'Jul-21', 'Aug-21', 'Sep-21',
       'Oct-21', 'Nov-21', 'Dec-21', '2023-04', '2023-05'], dtype=object)

In [8]:
import functools # not required, but helps in production
def unpack_df_columns(func):
    """
    A general use decorator to unpack a df[subset] of columns
    into a function which expects the values at those columns
    as arguments
    """
    
    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):
        
        # args[0] is a pandas series equal in length as the 
        # df[subset] to which the apply function is applied 
        series = args[0]

        # series.values holds the number of arguments expected
        # by func and is os length len(df[subset].columns)
        return func(*series.values)

    return _unpack_df_columns

In [9]:
def date_change(val):
    if val == 'Jul-22':
        return '2022-07'
    elif val == 'Jun-22':
        return '2022-06'
    elif val == 'May-22':
        return '2022-05'
    elif val == 'Apr-22':
        return '2022-04'
    elif val == 'Mar-22':
        return '2022-03'
    elif val == 'Feb-22':
        return '2022-02'
    elif val == 'Jan-22':
        return '2022-01'
    
    elif val == 'Jan-21':
        return '2021-01'
    elif val == 'Feb-21':
        return '2021-02'
    elif val == 'Mar-21':
        return '2021-03'
    elif val == 'Apr-21':
        return '2021-04'
    elif val == 'May-21':
        return '2021-05'
    elif val == 'Jun-21':
        return '2021-06'
    elif val == 'Jul-21':
        return '2021-07'
    elif val == 'Aug-21':
        return '2021-08'
    elif val == 'Sep-21':
        return '2021-09'
    elif val == 'Oct-21':
        return '2021-10'
    elif val == 'Nov-21':
        return '2021-11'
    elif val == 'Dec-21':
        return '2021-12'
    
    else:
        return val

In [10]:
def 월평균가입자수(가입자수):
    return int(가입자수/12)

In [11]:
@unpack_df_columns
def get_평균소득월액(고지액, 인원):
    평균소득월액 = (int(고지액) / int(인원)) / 0.09
    return int(평균소득월액)

In [12]:
# df1 = df.copy()
# df1.info()

## 경쟁사 Data Load

In [13]:
comp_df = pd.read_excel("./company.xlsx")
comp_df

,사업장명,사업자등록번호,약식명
0,현대자동차(주),101810,현대차
1,현대모비스(주),101811,모비스
2,삼성전자(주),124810,삼성전자
3,두산밥캣주식회사,201863,두산밥캣
4,볼보그룹코리아(주),609813,볼보코리아
5,볼보그룹코리아(주)건설기계(서울),120851,볼보코리아
6,두산산업차량주식회사,802880,두산산차
7,(주)대동,514810,대동
8,(주)한양정밀,131810,한양정밀
9,(주)모트롤,263810,모트롤


In [14]:
# comp_df["사업장명"].tolist()

In [16]:
df = df.loc[(df["사업장명"].isin(comp_df["사업장명"].tolist())) & (df["사업자등록번호"].isin(comp_df["사업자등록번호"].tolist()))]

In [17]:
df = pd.merge(df,comp_df, on=['사업장명','사업자등록번호'], how='left')

## 날짜 변환

In [18]:
df['자료생성년월'] = df['자료생성년월'].apply(date_change)

In [19]:
df['자료생성년월'].unique()

array(['2023-01', '2022-12', '2022-11', '2022-10', '2022-09', '2022-08',
       '2022-07', '2022-06', '2022-05', '2022-04', '2022-03', '2022-02',
       '2022-01', '2023-02', '2023-03', '2021-01', '2021-02', '2021-03',
       '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09',
       '2021-10', '2021-11', '2021-12', '2023-04', '2023-05'],
      dtype=object)

## 그룹핑

In [20]:
gdf = df.groupby(['약식명', '자료생성년월'])[['가입자수', '신규취득자수', '상실가입자수', '당월고지금액']].sum().reset_index()

In [21]:
gdf

,약식명,자료생성년월,가입자수,신규취득자수,상실가입자수,당월고지금액
0,HCE,2021-01,1355,7,14,567216740
1,HCE,2021-02,1374,33,12,576385980
2,HCE,2021-03,1364,2,10,572718340
3,HCE,2021-04,1391,37,10,579448720
4,HCE,2021-05,1390,9,15,578930280
...,...,...,...,...,...,...
357,현대차,2023-01,68104,1227,681,32113238560
358,현대차,2023-02,68410,987,673,32167723920
359,현대차,2023-03,68286,549,657,32079753340
360,현대차,2023-04,68413,784,567,32044180080


In [22]:
gdf.columns = ["약식명", "기준일자", "직원수", "신규취득", "자격상실", "당월고지금액"]

In [23]:
gdf["평균소득월액"] = gdf[["당월고지금액","직원수"]].astype('str').apply(get_평균소득월액, axis=1)

In [24]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   약식명     362 non-null    object
 1   기준일자    362 non-null    object
 2   직원수     362 non-null    int64 
 3   신규취득    362 non-null    int64 
 4   자격상실    362 non-null    int64 
 5   당월고지금액  362 non-null    int64 
 6   평균소득월액  362 non-null    int64 
dtypes: int64(5), object(2)
memory usage: 19.9+ KB


In [25]:
gdf.head()

,약식명,기준일자,직원수,신규취득,자격상실,당월고지금액,평균소득월액
0,HCE,2021-01,1355,7,14,567216740,4651223
1,HCE,2021-02,1374,33,12,576385980,4661054
2,HCE,2021-03,1364,2,10,572718340,4665349
3,HCE,2021-04,1391,37,10,579448720,4628554
4,HCE,2021-05,1390,9,15,578930280,4627740


In [26]:
요약통계 = gdf.groupby(['약식명'])[['직원수', '신규취득', '자격상실', '평균소득월액']].mean().round().reset_index()
요약통계

,약식명,직원수,신규취득,자격상실,평균소득월액
0,HCE,1373.0,20.0,23.0,4867923.0
1,HDI,2705.0,29.0,34.0,5093248.0
2,HDX,432.0,38.0,4.0,5115390.0
3,대동,1191.0,46.0,38.0,4397579.0
4,두산밥캣,130.0,4.0,3.0,5105710.0
5,두산산차,601.0,37.0,9.0,4760331.0
6,모비스,10839.0,109.0,61.0,5074958.0
7,모트롤,497.0,5.0,5.0,5083398.0
8,볼보코리아,1435.0,12.0,11.0,4972545.0
9,삼성전자,113324.0,1025.0,512.0,5173245.0


In [27]:
gdf['기준일자'].unique()

array(['2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
       '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
       '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
       '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
       '2023-01', '2023-02', '2023-03', '2023-04', '2023-05'],
      dtype=object)

In [28]:
import pickle
with open ("pickle_df2.pickle", 'wb') as pickle_filename:
    pickle.dump(gdf, pickle_filename)

In [29]:
import plotly.express as px
df = px.data.gapminder().query("continent == 'Oceania'")
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
60,Australia,Oceania,1952,69.120,8691212,10039.59564,AUS,36
61,Australia,Oceania,1957,70.330,9712569,10949.64959,AUS,36
62,Australia,Oceania,1962,70.930,10794968,12217.22686,AUS,36
63,Australia,Oceania,1967,71.100,11872264,14526.12465,AUS,36
64,Australia,Oceania,1972,71.930,13177000,16788.62948,AUS,36
65,Australia,Oceania,1977,73.490,14074100,18334.19751,AUS,36
66,Australia,Oceania,1982,74.740,15184200,19477.00928,AUS,36
67,Australia,Oceania,1987,76.320,16257249,21888.88903,AUS,36
68,Australia,Oceania,1992,77.560,17481977,23424.76683,AUS,36
69,Australia,Oceania,1997,78.830,18565243,26997.93657,AUS,36
